# Preprocessing

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path


def rename_columns(df):
    # Patient Characteristics columns
    patient_characteristics = [
        "Patient number", "Age (Continuos)", "Sex (Male/female)", "BMI (Continuous",
        "Roken (Yes/No)", "Diabetes (Yes/No)", "Cardiovascular disease (Yes/No)",
        "Rheumatoid Arthritis(Yes/No)"
    ]

    # Fracture Characteristics columns
    fracture_characteristics = [
        "Treatment (Conservative/Operative)",
        "High Energy trauma (Yes/No)",
        "Trauma mechanism (Valgus-flexion, Valgus-extension, Valgus-hyperextension, Varus-flexion, Varus-extension, "
        "Varus-Hyperextension)",
        "Side Fracture fractuur (Right/Left)", "AO/OTA Classification ( 1-6)",
        "Posterior involvement (Yes/No)", "Fracture fibula head (Yes/No)",
        "Cruris Fracture (Yes/No)", "2D gap (Continuous, mm)", "2D Step-off (Continuous, mm)", "3DGap Area",
        "AO/OTA - Severtity"
    ]

    # Postop Characteristics columns
    postop_characteristics = [
        "Condylar width (Continuous, mm)", "Incongruence (Continuous, mm)",
        "MPTA(Continuous, degree)", "PPTA (Continuous, degree)",
        "Revision surgery (Yes/No)", "Complicaties (Yes/No)", "MPTA (1=wrong,0=good)",
        "PPTA  (1=wrong,0=good)"
    ]

    # Outcome Values columns
    outcome_values = [
        "Total Knee Prosthesis within 2 yrs (Yes/No)", "Total Knee Prosthesis within 10 yrs (Yes/No)",
        "Symptoms (Continuous, 0-100)", "Pain (Continuous, 0-100)", "ADL (Continuous, 0-100)",
        "Sports (Continuous, 0-100)", "Quality of Life (Continuous, 0-100)", "VAS-Satisfaction (1-10) ",
        "Total Knee Prosthesis within 5 yrs (Yes/No)"
    ]

    raw_column_names = df.iloc[0]
    new_column_names = raw_column_names
    for i, column in enumerate(raw_column_names):
        new_column = ""

        if column in patient_characteristics:
            new_column = "patient_" + column
        elif column in fracture_characteristics:
            new_column = "fracture_" + column
        elif column in postop_characteristics:
            new_column = "postop_" + column
        elif column in outcome_values:
            new_column = "outcome_" + column
        else:
            raise ValueError(f"Found unknown column in dataset: {column}, have the column names been modified "
                             f"or have new columns been added?")

        new_column_names[i] = new_column

    df.columns = new_column_names
    df = df.tail(-1)
    return df


def update_data_types(df):
    type_changes = {
        "patient_Age (Continuos)": "float64",
        "patient_Sex (Male/female)": "category",
        "patient_BMI (Continuous": "float64",
        "fracture_Treatment (Conservative/Operative)": "category",
        "fracture_Trauma mechanism (Valgus-flexion, Valgus-extension, Valgus-hyperextension, Varus-flexion, Varus-extension, Varus-Hyperextension)": "category",
        "fracture_Side Fracture fractuur (Right/Left)": "category",
        "fracture_AO/OTA Classification ( 1-6)": "category",
        "fracture_AO/OTA - Severtity": "category",
        "fracture_2D gap (Continuous, mm)": "float64",
        "fracture_2D Step-off (Continuous, mm)": "float64",
        "fracture_3DGap Area": "float64",
        "postop_Condylar width (Continuous, mm)": "float64",
        "postop_Incongruence (Continuous, mm)": "float64",
        "postop_MPTA(Continuous, degree)": "float64",
        "postop_PPTA (Continuous, degree)": "float64",
        "postop_MPTA (1=wrong,0=good)":"bool",
        "postop_PPTA  (1=wrong,0=good)":"bool",
        "patient_Roken": "bool",
        "patient_Diabetes": "bool",
        "patient_Cardiovascular_Disease": "bool",
        "patient_Rheumatoid_Arthritis": "bool",
        "fracture_High_Energy_trauma": "bool",
        "postop_Revision_Surgery": "bool",
        "outcome_Total_Knee_Prothesis_Within_2_Yrs": "bool",
        "outcome_Total_Knee_Prothesis_Within_5_Yrs": "bool",
        "outcome_Total_Knee_Prothesis_Within_10_Yrs": "bool"
    }

    for col, dtype in type_changes.items():
        if dtype == "float64":
            df[col] = pd.to_numeric(df[col], errors='coerce')
        elif dtype == "bool":
            df[col] = df[col].astype("bool")
        elif dtype == "category":
            # Attempting to convert to category. If conversion fails, the value will remain as it was.
            try:
                df[col] = df[col].str.strip()
                # Replace multiple spaces with a single space
                df[col] = df[col].str.replace(r'\s+', ' ')
                df[col] = df[col].astype('category')
            except AttributeError:
                pass
    return df


def update_column(df, column, new_name, func, *args, **kwargs):
    df[new_name] = func(df[column], *args, **kwargs)
    df.drop(columns=[column], inplace=True)
    return df


def normalize_data(df):
    bool_replacement = {"Yes": True, "No": False, "no": False}
    replace_func = lambda c, rep_dict: c.replace(rep_dict)

    update_column(df, 'patient_Roken (Yes/No)', 'patient_Roken', replace_func,
                  bool_replacement)
    update_column(df, 'patient_Diabetes (Yes/No)', 'patient_Diabetes', replace_func,
                  bool_replacement)
    update_column(df, 'patient_Cardiovascular disease (Yes/No)', 'patient_Cardiovascular_Disease',
                  replace_func, bool_replacement)
    update_column(df, 'patient_Rheumatoid Arthritis(Yes/No)', 'patient_Rheumatoid_Arthritis',
                  replace_func, bool_replacement)

    update_column(df, 'fracture_High Energy trauma (Yes/No)', 'fracture_High_Energy_trauma',
                  replace_func, bool_replacement)
    update_column(df, 'fracture_Posterior involvement (Yes/No)', 'fracture_Posterior_Involvement',
                  replace_func, bool_replacement)
    update_column(df, 'fracture_Fracture fibula head (Yes/No)', 'fracture_Fracture_Fibula_Head',
                  replace_func, bool_replacement)
    update_column(df, 'fracture_Cruris Fracture (Yes/No)', 'fracture_Cruris_Fracture',
                  replace_func, bool_replacement)

    update_column(df, 'postop_Revision surgery (Yes/No)', 'postop_Revision_Surgery',
                  replace_func, bool_replacement)
    update_column(df, 'postop_Complicaties (Yes/No)', 'postop_Complicaties', replace_func,bool_replacement)

    update_column(df, 'outcome_Total Knee Prosthesis within 2 yrs (Yes/No)',
                  'outcome_Total_Knee_Prothesis_Within_2_Yrs', replace_func,bool_replacement)
    update_column(df, 'outcome_Total Knee Prosthesis within 5 yrs (Yes/No)',
                  'outcome_Total_Knee_Prothesis_Within_5_Yrs', replace_func,bool_replacement)
    update_column(df, 'outcome_Total Knee Prosthesis within 10 yrs (Yes/No)',
                  'outcome_Total_Knee_Prothesis_Within_10_Yrs', replace_func,bool_replacement)

    return df

def preprocess(data_dir="data", filename="Features-Filled-V3.xlsx"):
    filepath = Path(data_dir).joinpath(filename)
    df = pd.read_excel(filepath)

    df = rename_columns(df)
    df = normalize_data(df)
    df = update_data_types(df)

    preprocessed_path = Path(data_dir).joinpath("preprocessed.csv")
    df.to_csv(preprocessed_path, index=False)


if __name__ == '__main__':
    preprocess()

# Training and Evaluation

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (roc_curve, auc, f1_score, confusion_matrix,
                             precision_score, recall_score, ConfusionMatrixDisplay)
from sklearn.calibration import calibration_curve
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
import pickle
from patsy import dmatrix

# Load the dataset
file_path = 'preprocessed.csv'
df = pd.read_csv(file_path)

center_column = 'patient_Hospital'  # Adjust as needed
unique_centers = df[center_column].unique()

# Define features
continuous_features = [
    "fracture_2D gap (Continuous, mm)",
    "fracture_2D Step-off (Continuous, mm)",
    "patient_Age (Continuos)",
    "patient_BMI (Continuous"
]

categorical_features = [
    "patient_Sex (Male/female)",
    "fracture_AO/OTA Classification ( 1-6)",
    "postop_MPTA (1=wrong,0=good)",
    "postop_PPTA  (1=wrong,0=good)"
]

# Create an 'output' folder (at the top level) if it doesn't exist
os.makedirs("output", exist_ok=True)

# We'll store outcomes.csv here
outcomes_path = os.path.join("output", "outcomes.csv")

years = [2, 5]
models = ['logistic', 'forest', 'xgboost']

for year in years:
    target = f"outcome_Total_Knee_Prothesis_Within_{year}_Yrs"

    # Drop rows with missing target
    df_year = df.dropna(subset=[target])
    y = df_year[target].astype(bool)

    for model_name in models:
        print(f"\nModel: {model_name}, Year: {year}")

        # --------------------------------------------------
        # Create a subfolder inside 'output' for this run
        # e.g. "output/logistic_2"
        # --------------------------------------------------
        run_folder = os.path.join("output", f"{model_name}_{year}")
        os.makedirs(run_folder, exist_ok=True)

        # Lists for metrics
        all_f1_scores = []
        all_aucs = []
        all_precision = []
        all_recall = []

        # -- NEW: For macro metrics
        all_f1_macro = []
        all_precision_macro = []
        all_recall_macro = []

        cumulative_confusion_matrix = np.zeros((2, 2))
        all_probs = []
        all_y_test = []
        all_y_pred = []

        # We'll collect patient-level predictions here,
        # then write them out to predictions.csv at the end of this model-year loop.
        predictions_list_for_model_year = []

        # -- NEW: We'll collect feature importances (per fold) here
        feature_importances_list_for_model_year = []

        # -----------------------------------------------------------------
        # ADDED: Lists to store each fold's ROC for the multi‐fold plot
        # -----------------------------------------------------------------
        fold_fprs = []
        fold_tprs = []
        fold_roc_aucs = []

        # External validation by leaving one center out
        for test_center in unique_centers:
            train_mask = df_year[center_column] != test_center
            test_mask  = df_year[center_column] == test_center

            # Split data
            X_train_df, X_test_df = df_year.loc[train_mask], df_year.loc[test_mask]
            y_train, y_test       = y[train_mask], y[test_mask]

            if len(y_test) == 0:
                continue

            # -------------------------------------------------
            # 1. Split continuous vs. categorical in training
            # -------------------------------------------------
            X_train_cont = X_train_df[continuous_features].copy()
            X_test_cont  = X_test_df[continuous_features].copy()

            X_train_cat = X_train_df[categorical_features].copy()
            X_test_cat  = X_test_df[categorical_features].copy()

            # -------------------------------------------------
            # 2. Impute missing in continuous, then scale
            #    (fit only on training)
            # -------------------------------------------------
            imputer = KNNImputer(n_neighbors=5)
            X_train_cont_imputed = imputer.fit_transform(X_train_cont)
            X_test_cont_imputed  = imputer.transform(X_test_cont)

            scaler = StandardScaler()
            X_train_cont_scaled = scaler.fit_transform(X_train_cont_imputed)
            X_test_cont_scaled  = scaler.transform(X_test_cont_imputed)

            # Turn back into DataFrames so we can reference columns
            X_train_cont_scaled = pd.DataFrame(X_train_cont_scaled,
                                               columns=continuous_features,
                                               index=X_train_df.index)
            X_test_cont_scaled = pd.DataFrame(X_test_cont_scaled,
                                              columns=continuous_features,
                                              index=X_test_df.index)

            if model_name == 'logistic':
                # -------------------------------------------------
                # 3. Create splines from *scaled* continuous data
                # -------------------------------------------------
                X_spline_list_train = []
                X_spline_list_test = []

                # We'll track the features actually used in splines
                used_spline_feats = []

                for cfeat in continuous_features:
                    cvals_train = X_train_cont_scaled[cfeat].dropna()
                    # If almost no data remains, skip feature
                    if len(cvals_train) < 4:
                        print(f"Dropping {cfeat} due to insufficient data.")
                        continue

                    # Generate knots from scaled training
                    knots = np.percentile(cvals_train, [25, 50, 75])

                    # Create spline basis for training set
                    spline_train = dmatrix(
                        "bs(x, knots=({},{},{}), degree=3, include_intercept=False)".format(*knots),
                        {"x": X_train_cont_scaled[cfeat]},
                        return_type='dataframe'
                    )
                    spline_col_names = [f"{cfeat}_spline_{i}" for i in range(spline_train.shape[1])]
                    spline_train.columns = spline_col_names
                    X_spline_list_train.append(spline_train)

                    # Replicate same spline transformations in test
                    spline_test = dmatrix(
                        "bs(x, knots=({},{},{}), degree=3, include_intercept=False)".format(*knots),
                        {"x": X_test_cont_scaled[cfeat]},
                        return_type='dataframe'
                    )
                    spline_test.columns = spline_col_names
                    X_spline_list_test.append(spline_test)

                    used_spline_feats.append(cfeat)

                # Combine all spline features
                if len(X_spline_list_train) > 0:
                    X_spline_all_train = pd.concat(X_spline_list_train, axis=1)
                    X_spline_all_test  = pd.concat(X_spline_list_test, axis=1)
                else:
                    # If no valid spline features remain
                    X_spline_all_train = pd.DataFrame(index=X_train_df.index)
                    X_spline_all_test  = pd.DataFrame(index=X_test_df.index)

                # -------------------------------------------------
                # 4. Combine (scaled) spline features with cat
                # -------------------------------------------------
                X_train_final = pd.concat([X_train_cat, X_spline_all_train], axis=1)
                X_test_final  = pd.concat([X_test_cat,  X_spline_all_test],  axis=1)

            else:
                # For non-logistic, we just use the scaled numeric + cat
                X_train_final = pd.concat([X_train_cont_scaled, X_train_cat], axis=1)
                X_test_final  = pd.concat([X_test_cont_scaled,  X_test_cat],  axis=1)

            print(f"train_columns: {X_train_final.columns}", len(X_train_final.columns))

            # -------------------------------------------------
            # 5. Now we have X_train_final, X_test_final ready
            #    Train model, do threshold search, evaluate
            # -------------------------------------------------
            # Model selection
            if model_name == 'logistic':
                base_model = LogisticRegression()
            elif model_name == 'forest':
                base_model = RandomForestClassifier()
            elif model_name == 'xgboost':
                base_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

            # Nested validation for threshold selection
            X_train_nested, X_val_nested, y_train_nested, y_val_nested = train_test_split(
                X_train_final, y_train, test_size=0.2, random_state=42
            )
            model = base_model
            model.fit(X_train_nested, y_train_nested)

            thresholds = np.linspace(0.1, 0.9, 9)
            y_proba_val = model.predict_proba(X_val_nested)[:, 1]
            best_threshold = 0
            best_f1 = 0
            for threshold in thresholds:
                y_pred_threshold = (y_proba_val >= threshold).astype(int)
                f1_tmp = f1_score(y_val_nested, y_pred_threshold)
                if f1_tmp > best_f1:
                    best_f1 = f1_tmp
                    best_threshold = threshold

            # Retrain on full training set
            model.fit(X_train_final, y_train)
            y_proba = model.predict_proba(X_test_final)[:, 1]
            y_pred = (y_proba >= best_threshold).astype(int)

            # Metrics (binary average)
            f1       = f1_score(y_test, y_pred)
            precision= precision_score(y_test, y_pred)
            recall   = recall_score(y_test, y_pred)

            # -- NEW: macro-averaged metrics
            f1_macro       = f1_score(y_test, y_pred, average='macro')
            precision_macro= precision_score(y_test, y_pred, average='macro')
            recall_macro   = recall_score(y_test, y_pred, average='macro')

            fpr, tpr, _ = roc_curve(y_test, y_proba)
            roc_auc     = auc(fpr, tpr)

            all_f1_scores.append(f1)
            all_aucs.append(roc_auc)
            all_precision.append(precision)
            all_recall.append(recall)

            all_f1_macro.append(f1_macro)
            all_precision_macro.append(precision_macro)
            all_recall_macro.append(recall_macro)

            all_probs.extend(y_proba)
            all_y_test.extend(y_test)
            all_y_pred.extend(y_pred)

            fold_confusion_matrix = confusion_matrix(y_test, y_pred)
            cumulative_confusion_matrix += fold_confusion_matrix

            # --------------------------------------------
            # Collect patient-level predictions for output
            # --------------------------------------------
            df_pred_center = pd.DataFrame({
                "patient_Patient number": X_test_df["patient_Patient number"],  # adapt if needed
                "center": test_center,
                "year": year,
                "model": model_name,
                "true_label": y_test.values,
                "predicted_proba": y_proba,
                "predicted_label": y_pred
            })
            predictions_list_for_model_year.append(df_pred_center)

            # --------------------------------------------
            # Collect feature importances for this fold
            # --------------------------------------------
            if model_name == 'logistic':
                importances = model.coef_[0]  # shape: (n_features,)
            else:
                importances = model.feature_importances_  # shape: (n_features,)

            final_feature_names = X_train_final.columns.tolist()

            df_importance = pd.DataFrame({
                'feature': final_feature_names,
                'importance': importances
            })
            df_importance['center'] = test_center
            df_importance['model'] = model_name
            df_importance['year'] = year

            feature_importances_list_for_model_year.append(df_importance)

            # --------------------------------------------
            # ADDED: Save fold's ROC data for the final plot
            # --------------------------------------------
            fold_fprs.append(fpr)
            fold_tprs.append(tpr)
            fold_roc_aucs.append(roc_auc)

        # End of center loop

        # -------------------------------
        # ADDED: Plot the multi‐fold ROC
        # -------------------------------
        if len(fold_fprs) > 0:
            plt.figure()

            # We'll use a matplotlib color map, picking one color per fold.
            cmap = plt.cm.get_cmap('viridis')
            colors_list = [cmap(i / (len(fold_fprs) - 1))
                          for i in range(len(fold_fprs))]

            # Plot each fold with its own color & label
            for i, (fpr_i, tpr_i) in enumerate(zip(fold_fprs, fold_tprs)):
                plt.plot(
                    fpr_i,
                    tpr_i,
                    color=colors_list[i],
                    alpha=0.6,  # some transparency so they don't overwhelm
                    label=f'Fold {i} (AUC = {fold_roc_aucs[i]:.2f})'
                )

            # Interpolate all TPRs to a common FPR axis
            mean_fpr = np.linspace(0, 1, 100)
            tprs_interp = []
            for fpr_i, tpr_i in zip(fold_fprs, fold_tprs):
                tpr_interp = np.interp(mean_fpr, fpr_i, tpr_i)
                tpr_interp[0] = 0.0
                tprs_interp.append(tpr_interp)

            mean_tpr = np.mean(tprs_interp, axis=0)
            std_tpr = np.std(tprs_interp, axis=0)
            mean_auc_folds = np.mean(fold_roc_aucs)
            std_auc_folds  = np.std(fold_roc_aucs)

            # Plot mean ROC in dark blue
            plt.plot(
                mean_fpr,
                mean_tpr,
                color='blue',
                linewidth=2,
                label=r'Mean ROC (AUC = %.2f $\pm$ %.2f)' % (mean_auc_folds, std_auc_folds)
            )

            # Shade ±1 std. dev. around the mean
            plt.fill_between(
                mean_fpr,
                mean_tpr - std_tpr,
                mean_tpr + std_tpr,
                color='grey',
                alpha=0.2,
                label=r'$\pm$ 1 std. dev.'
            )

            # Chance line (red, dashed)
            plt.plot([0, 1], [0, 1], linestyle='--', color='red', label='Chance')

            plt.xlabel('False Positive Rate (Positive label: True)')
            plt.ylabel('True Positive Rate (Positive label: True)')
            plt.title('Receiver Operating Characteristic')
            plt.legend(loc='lower right')

            # Save the figure to the current model-year folder
            plt.savefig(os.path.join(run_folder, "roc_curves.png"))
            plt.show()

        # ---------------------------------------------
        # Print average results across centers (existing code)
        # ---------------------------------------------
        if len(all_f1_scores) > 0:
            mean_f1       = np.mean(all_f1_scores)
            mean_auc      = np.mean(all_aucs)
            mean_precision= np.mean(all_precision)
            mean_recall   = np.mean(all_recall)

            mean_f1_macro       = np.mean(all_f1_macro)
            mean_precision_macro= np.mean(all_precision_macro)
            mean_recall_macro   = np.mean(all_recall_macro)

            print(f"Mean F1: {mean_f1:.2f}")
            print(f"Mean AUC: {mean_auc:.2f}")
            print(f"Mean Precision: {mean_precision:.2f}")
            print(f"Mean Recall: {mean_recall:.2f}")

            # NEW: print macro metrics
            print(f"Mean F1 (macro): {mean_f1_macro:.2f}")
            print(f"Mean Precision (macro): {mean_precision_macro:.2f}")
            print(f"Mean Recall (macro): {mean_recall_macro:.2f}")

            # -----------------------------------------
            # Confusion matrix & store in local folder
            # -----------------------------------------
            ConfusionMatrixDisplay(confusion_matrix=cumulative_confusion_matrix).plot(values_format='g')
            plt.title(f"Confusion Matrix: {model_name}, {year}-Year")
            plt.savefig(os.path.join(run_folder, "confusion_matrix.png"))
            plt.show()

            # Bootstrapping for AUC variability
            n_bootstraps = 100
            rng = np.random.RandomState(42)
            boot_aucs = []
            all_y_test_arr = np.array(all_y_test)
            all_probs_arr  = np.array(all_probs)
            for i in range(n_bootstraps):
                indices = rng.randint(0, len(all_y_test_arr), len(all_y_test_arr))
                # If the sample is all one class, skip
                if len(np.unique(all_y_test_arr[indices])) < 2:
                    continue
                fpr_b, tpr_b, _ = roc_curve(all_y_test_arr[indices], all_probs_arr[indices])
                roc_auc_b = auc(fpr_b, tpr_b)
                boot_aucs.append(roc_auc_b)

            if len(boot_aucs) > 0:
                print(f"Bootstrap AUC Mean: {np.mean(boot_aucs):.2f}, "
                      f"95% CI: ({np.percentile(boot_aucs,2.5):.2f}-{np.percentile(boot_aucs,97.5):.2f})")

            # -------------------------------------------------
            # Calibration curve & store in local model folder
            # -------------------------------------------------
            prob_true, prob_pred = calibration_curve(all_y_test, all_probs, n_bins=10)
            plt.figure()
            plt.plot(prob_pred, prob_true, marker='o', label='Calibration')
            plt.plot([0,1],[0,1], 'k--', label='Perfectly calibrated')
            plt.title(f"Calibration Curve: {model_name}, {year}-Year")
            plt.xlabel('Predicted Probability')
            plt.ylabel('Fraction of Positives')
            plt.legend()
            plt.savefig(os.path.join(run_folder, "calibration_curve.png"))
            plt.show()

            # Summarize run data (for outcomes.csv in 'output' folder)
            run_data = {
                "name": f"{model_name}_{year}",
                "auc": round(mean_auc, 3),
                "f1": round(mean_f1, 3),
                "recall": round(mean_recall, 3),
                "precision": round(mean_precision, 3),
                # NEW: macro-averaged metrics
                "f1_macro": round(mean_f1_macro, 3),
                "recall_macro": round(mean_recall_macro, 3),
                "precision_macro": round(mean_precision_macro, 3),

                "bootstrap_auc_mean": round(np.mean(boot_aucs), 3) if len(boot_aucs) > 0 else np.nan,
                "bootstrap_auc_ci_lower": round(np.percentile(boot_aucs, 2.5), 3) if len(boot_aucs) > 0 else np.nan,
                "bootstrap_auc_ci_upper": round(np.percentile(boot_aucs, 97.5), 3) if len(boot_aucs) > 0 else np.nan,
            }

            # Append results to outcomes.csv in 'output/'
            try:
                outcome = pd.read_csv(outcomes_path)
                outcome = pd.concat([outcome, pd.DataFrame.from_dict([run_data])], ignore_index=True)
                outcome.to_csv(outcomes_path, index=False)
            except FileNotFoundError:
                pd.DataFrame.from_dict([run_data]).to_csv(outcomes_path, index=False)

            # --------------------------------------------
            # Store predictions for this model+year folder
            # --------------------------------------------
            if len(predictions_list_for_model_year) > 0:
                df_all_preds_this_model_year = pd.concat(predictions_list_for_model_year, ignore_index=True)
                df_all_preds_this_model_year.to_csv(
                    os.path.join(run_folder, "predictions.csv"),
                    index=False
                )

            # -------------------------------------------------------
            # Store feature importances across folds in this run folder
            # -------------------------------------------------------
            if len(feature_importances_list_for_model_year) > 0:
                df_feature_importances = pd.concat(feature_importances_list_for_model_year, ignore_index=True)
                df_feature_importances.to_csv(
                    os.path.join(run_folder, "feature_importances.csv"),
                    index=False
                )

            # ------------------------------------------------
            # Always save model & scaler in current run folder
            # ------------------------------------------------
            model_filename = os.path.join(run_folder, "model.sav")
            pickle.dump(model, open(model_filename, 'wb'))

            # Save the scaler (only relevant to continuous features)
            scaler_filename = os.path.join(run_folder, "scaler.sav")
            pickle.dump(scaler, open(scaler_filename, 'wb'))
